<a href="https://colab.research.google.com/github/deep200315/RAG-Model-for-QA-Bot/blob/main/RAG.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%pip install --upgrade --quiet  langchain langchain-community langchainhub langchain-openai langchain-pinecone

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 817.7/817.7 kB 4.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 25.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 299.3/299.3 kB 27.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.0/116.0 kB 13.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 311.6/311.6 kB 29.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 71.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 215.9/215.9 kB 23.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 3.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.0/53.0 kB 4.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 141.1/141.1 kB 14.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 4.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 8.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━

In [ ]:
#loading data
from langchain.document_loaders import TextLoader

loader = TextLoader('/content/drive/MyDrive/Wazirx/prep_data.txt')
documents = loader.load()

In [ ]:
len(documents[0].page_content)


78022

In [ ]:
#creating chunks of data
from langchain_text_splitters import RecursiveCharacterTextSplitter
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000, chunk_overlap=100, add_start_index=True
)
all_splits = text_splitter.split_documents(documents)

In [ ]:
len(all_splits)


87

In [ ]:
len(all_splits[0].page_content)


997

In [ ]:
use_serverless = True


In [ ]:
all_splits[10].metadata

{'source': '/content/drive/MyDrive/Wazirx/prep_data.txt', 'start_index': 9000}

In [ ]:
from pinecone import Pinecone, ServerlessSpec, PodSpec
import time
from google.colab import userdata
# configure client
pc = Pinecone(api_key=userdata.get("PINECONE_API_KEY"))
if use_serverless:
    spec = ServerlessSpec(cloud='aws', region='us-east-1')
else:
    # if not using a starter index, you should specify a pod_type too
    spec = PodSpec()
# check for and delete index if already exists
index_name = 'langchain-retrieval-augmentation-fast'
if index_name in pc.list_indexes().names():
    pc.delete_index(index_name)
# create a new index
pc.create_index(
    index_name,
    dimension=1536,  # dimensionality of text-embedding-ada-002
    metric='dotproduct',
    spec=spec
)
# wait for index to be initialized
while not pc.describe_index(index_name).status['ready']:
    time.sleep(1)


In [ ]:
#storing vectors in pinecone with OpenAIEmbeddings
from langchain_pinecone import PineconeVectorStore
from langchain_openai import OpenAIEmbeddings
from google.colab import userdata
open_ai_api_key=userdata.get('OPEN_AI_API_KEY')
pinecone_api_key=userdata.get('PINECONE_API_KEY')
# Specify the index name
index_name = "langchain-retrieval-augmentation-fast"
embeddings = OpenAIEmbeddings(openai_api_key=open_ai_api_key)
# Initialize the PineconeVectorStore with the index name
vectorstore = PineconeVectorStore(index_name, embeddings)



In [ ]:
import os
import getpass

os.environ["PINECONE_API_KEY"]=getpass.getpass()

··········


In [ ]:
vectorstore_from_docs = PineconeVectorStore.from_documents(
        all_splits,
        index_name=index_name,
        embedding=embeddings
    )

In [ ]:
#1st method for querying
query = "What is supermarket"
docs = vectorstore_from_docs.similarity_search(query)
print(docs[0].page_content)

what is a supermarket business a supermarket is a large retail store that sells a wide variety of food and household products organized into aisles and sections for easy shopping what are the key components of a supermarket business the key components include inventory management customer service marketing and promotions employee management financial management and store operations how do supermarkets manage their inventory supermarkets use inventory management systems to track stock levels reorder products as needed and minimize waste they often use barcode scanning and rfid technology for accurate tracking what role does customer service play in a supermarket business customer service is crucial for providing assistance to shoppers resolving issues or complaints and ensuring a positive shopping experience friendly and knowledgeable staff can enhance customer satisfaction and loyalty how do supermarkets attract customers supermarkets attract customers through advertising promotions


In [ ]:
#2nd method for querying
retriever = vectorstore_from_docs.as_retriever(search_type="similarity", search_kwargs={"k": 6})

In [ ]:
retrieved_docs = retriever.invoke("What are business stratigies")

In [ ]:
len(retrieved_docs)

6

In [ ]:
print(retrieved_docs[0].page_content)

how do supermarkets attract customers supermarkets attract customers through advertising promotions loyalty programs and strategic placement of products they may also offer competitive pricing highquality products and convenient services such as online shopping and home delivery what marketing strategies do supermarkets use supermarkets use various marketing strategies such as flyers newspaper ads social media promotions and instore signage to promote sales and special offers they may also partner with suppliers for comarketing opportunities how do supermarkets manage their employees supermarkets manage their employees through hiring training scheduling and performance management they prioritize safety provide opportunities for advancement and offer benefits such as healthcare and retirement plans what financial considerations are important for supermarket businesses financial considerations include budgeting cash flow management pricing strategies and profitability analysis


In [ ]:
!pip install -qU langchain-openai

In [ ]:
import getpass
import os

os.environ["OPENAI_API_KEY"] = getpass.getpass()

··········


In [ ]:
#creating model
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-3.5-turbo-0125")

In [ ]:
#prompting and chainalising the model
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough

# Define the template for the prompt
template = """Use the following pieces of context to answer the question at the end.
If you don't know the answer, just say that you don't know, don't try to make up an answer.
Use three sentences maximum and keep the answer as concise as possible.
Always say "thanks for asking!" at the end of the answer.

Context: {context}

Question: {question}

Helpful Answer:"""

# Create a PromptTemplate object
prompt = PromptTemplate(
    template=template,
    input_variables=["context", "question"]
)

# Define the components of the processing pipeline
rag_chain = (
    {"context": retriever, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

# Get user input
user_input = input("Ask me about Supermaket Business: \n\n")

# Invoke the processing pipeline
result = rag_chain.invoke(user_input)
print("\nAnswer:\n\n",result)


Ask me about Supermaket Business: 

How do supermarkets manage pricing strategies to remain competitive while maintaining profitability?

Answer:

 Supermarkets analyze market trends, competitor pricing, and customer behavior to develop pricing strategies. They offer targeted promotions and optimize pricing to maximize sales and profitability while providing value to customers. Thanks for asking!
